# Keras - Tuner 

Keras Tuner est une bibliothèque qui vous aide à choisir l'ensemble optimal d'hyperparamètres pour votre programme TensorFlow.

On peut ajuster les hyperparamètres pour obtenir les valeurs qui peuvent aider à améliorer le modèle. Nous allons optimiser les hyperparamètres suivants dans le modèle :



     Nombre de couches cachés

     Nombre de neurones dans chaque couche cachée

     Taux d'apprentissage

     Fonction d'activation



Mais d'abord, nous devons installer le Keras Tuner. 

In [2]:
!pip install keras-tuner

In [3]:
from tensorflow import keras
from keras_tuner import RandomSearch

# Importation data 

On importe les packages nécessaires pour le deep learning. 

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img
from tensorflow.keras import layers
import matplotlib.pyplot as plt

On importe les données 

In [51]:
train_data_dir = 'C:/Users/na_to/OneDrive/Bureau/Insa/Mapromo/ML3/Exercices/Data/chihuahua-vs-muffin/train'
validation_data_dir = 'C:/Users/na_to/OneDrive/Bureau/Insa/Mapromo/ML3/Exercices/Data/chihuahua-vs-muffin/validation'
test_data_dir = 'C:/Users/na_to/OneDrive/Bureau/Insa/Mapromo/ML3/Exercices/Data/chihuahua-vs-muffin/test' 

On definit les images de taille (256,256)

In [52]:
image_size = (256, 256)
batch_size = 32  

train_ds = keras.preprocessing.image_dataset_from_directory(
    train_data_dir,
    image_size=image_size,
    batch_size=batch_size,
    label_mode = 'categorical'
)
val_ds = keras.preprocessing.image_dataset_from_directory(
    validation_data_dir,
    image_size=image_size,
    batch_size=batch_size,
    label_mode = 'categorical'
)

test_ds = keras.preprocessing.image_dataset_from_directory(
    test_data_dir,
    image_size=image_size,
    batch_size=batch_size,
    label_mode = 'categorical'
)

Found 568 files belonging to 2 classes.
Found 142 files belonging to 2 classes.
Found 16 files belonging to 2 classes.


On definit une variable qui contient le nom des classes : chihuahua et muffin

In [7]:
class_names = train_ds.class_names

On definit une variable qui prend en entrée (256,256,3)

In [8]:
img_height, img_width = image_size
input_shape = (img_height, img_width, 3)

On ajoute de la data augmentation dans notre réseau

In [9]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal", input_shape=input_shape),
        layers.RandomRotation(0.1),
    ]
)

In [10]:
train_ds = train_ds.prefetch(buffer_size=32)
val_ds = val_ds.prefetch(buffer_size=32)

# Optimisation

$\underline{Fonction\_d'optimisation}$ 

On définit la fonction $\textbf{build_model}$ pour construire un modèle de réseau de neurone de convolution où les hyperparamètres sont le nombre de neurones dans la couche cachée et le taux d'apprentissage 

In [23]:
def build_model(hp):          #hp signifie hyper parameters
    # Preprocessing
    model = Sequential() 
    model.add(data_augmentation)
    model.add(layers.Rescaling(1./255))
    model.add(Flatten(input_shape=(256,256,3)))
    #providing range for number of neurons in a hidden layer
    model.add(Dense(units=hp.Int('num_of_neurons',min_value=32,max_value=96,step=32),
                                    activation='relu'))
    #output layer
    model.add(Dense(2,activation='softmax'))
    #compiling the model
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2, 1e-3])),loss='categorical_crossentropy',metrics=['accuracy'])
    return model

Dans le code ci-dessus, nous avons défini la fonction par le nom build_model(hp) où hp signifie hyperparamètre. Lors de l'ajout de la couche cachée, nous utilisons la fonction hp.Int() qui prend la valeur entière et teste sur la plage spécifiée pour le réglage. Nous avons fourni la plage pour les neurones de 32 à 96 avec une taille de pas de 32 afin que le modèle teste sur les neurones 32, 64, 96.

Ensuite, nous avons ajouté la couche de sortie. Lors de la compilation du modèle, l'optimiseur Adam est utilisé avec différentes valeurs de taux d'apprentissage qui est le prochain hyperparamètre pour le réglage. La fonction hp.Choice() est utilisée pour tester l'une des deux valeurs fournies pour le taux d'apprentissage. 

In [32]:
#feeding the model and parameters to Random Search
tuner=RandomSearch(build_model,
    objective='val_accuracy',
    max_trials=3,
    executions_per_trial=3,
    directory='tuner1',
    project_name='Classification')

Le code ci-dessus utilise l'optimiseur d'hyperparamètres de recherche aléatoire. Les variables suivantes sont fournies à la recherche aléatoire. Le premier est le modèle, c'est-à-dire build_model, l'objectif suivant est val_accuracy, ce qui signifie que l'objectif du modèle est d'obtenir une bonne accuracy pour la validation. Ensuite, la valeur des trials et l'exécution par trial qui sont respectivement de 3 et 3 dans notre cas, ce qui signifie que 9 (3 * 3) itérations seront effectuées par le modèle pour trouver les meilleurs paramètres. Le répertoire et le nom du projet sont fournis pour enregistrer les valeurs de chaque essai. 

Une fois les valeurs enregistrées cela nous permet de ne plus re-fit le modèle et attendre comme la première exécution.

In [33]:
#this tells us how many hyperparameter we are tuning
#in our case it's 2 = neurons,learning rate
tuner.search_space_summary()

Search space summary
Default search space size: 2
num_of_neurons (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 96, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001], 'ordered': True}


In [34]:
epochs = 5 
tuner.search(train_ds.repeat(), epochs=epochs, steps_per_epoch=50, validation_data=val_ds)

Trial 3 Complete [00h 02m 45s]
val_accuracy: 0.6197183231512705

Best val_accuracy So Far: 0.8403755823771158
Total elapsed time: 00h 07m 38s
INFO:tensorflow:Oracle triggered exit


Le code ci-dessus exécute 3 trails avec 3 exécutions chacune et affichent les détails de trail qui fournissent la plus grande accuracy de validation. On peut voir la meilleure accuracy de validation obtenue par le modèle. 

Nous pouvons également vérifier le résumé de tous les sentiers effectués et les hyperparamètres choisis pour la meilleure accuracy en utilisant le code ci-dessous. La meilleure accuracy est obtenue en utilisant 32 neurones dans la couche cachée et 0,001 comme taux d'apprentissage. 

In [35]:
tuner.results_summary()

Results summary
Results in tuner1\Classification
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
num_of_neurons: 32
learning_rate: 0.001
Score: 0.8403755823771158
Trial summary
Hyperparameters:
num_of_neurons: 32
learning_rate: 0.01
Score: 0.8380281527837118
Trial summary
Hyperparameters:
num_of_neurons: 64
learning_rate: 0.01
Score: 0.6197183231512705


C'est ainsi que nous effectuons le réglage des réseaux de neurones à l'aide de Keras Tuner. 

$\underline{Amélioration\_de\_la\_fonction\_optimisation}$

On peut ajuster quelques paramètres supplémentaires dans le code suivant. Ici, nous fournissons également la plage du nombre de couches à utiliser dans le modèle qui se situe entre 2 et 10. 

In [54]:
def build_model_bis(hp):                 #hp signigie hyper parameters
    model_bis=Sequential()
    model_bis.add(data_augmentation)
    model_bis.add(layers.Rescaling(1./255))
    model_bis.add(Flatten(input_shape=(256,256,3)))
    #providing the range for hidden layers  
    for i in range(hp.Int('num_of_layers',2,10)):         
        #providing range for number of neurons in hidden layers
        model_bis.add(Dense(units=hp.Int('num_of_neurons'+ str(i),min_value=32,max_value=96,step=32),
                                    activation='relu'))
    model_bis.add(Dense(2,activation='softmax'))    #output layer
    #compiling the model
    model_bis.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2, 1e-3])),   #tuning learning rate
                  loss='categorical_crossentropy',metrics=['accuracy'])
    return model_bis

In [58]:
#feeding the model and parameters to Random Search
tuner_bis=RandomSearch(build_model_bis,
    objective='val_accuracy',
    max_trials=3,
    executions_per_trial=3,
    directory='tuner2',
    project_name='Classification')

Ici on a 3 hyperparamètres que nous réglons : nombres de couches, nombre de neurones et taux d'apprentissage

In [59]:
epochs = 5 
tuner_bis.search(train_ds.repeat(), epochs=epochs, steps_per_epoch=50, validation_data=val_ds)

Trial 3 Complete [00h 02m 47s]
val_accuracy: 0.52112677693367

Best val_accuracy So Far: 0.8192488352457682
Total elapsed time: 00h 09m 32s
INFO:tensorflow:Oracle triggered exit


On obtient 81% comme meilleur accuracy pour la validation 

In [60]:
tuner_bis.results_summary()

Results summary
Results in tuner2\Classification
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
num_of_layers: 2
num_of_neurons0: 64
num_of_neurons1: 32
learning_rate: 0.01
Score: 0.8192488352457682
Trial summary
Hyperparameters:
num_of_layers: 8
num_of_neurons0: 96
num_of_neurons1: 64
learning_rate: 0.01
num_of_neurons2: 32
num_of_neurons3: 32
num_of_neurons4: 32
num_of_neurons5: 32
num_of_neurons6: 32
num_of_neurons7: 32
Score: 0.8004694779713949
Trial summary
Hyperparameters:
num_of_layers: 3
num_of_neurons0: 32
num_of_neurons1: 96
learning_rate: 0.01
num_of_neurons2: 96
num_of_neurons3: 96
num_of_neurons4: 96
num_of_neurons5: 96
num_of_neurons6: 64
num_of_neurons7: 96
Score: 0.52112677693367


L'idéal dans ce cas serait de construire un réseau de neurone avec 2 couches :

    1ère couche : 64 neurones
    2ème couche : 32 neurones

On fixe ensuite comme valeur de taux d'apprentissage : 0.01

# Modèle avec les meilleurs hyperparamètres

In [61]:
model_test = Sequential()
# Preprocessing
model_test.add(data_augmentation)
model_test.add(layers.Rescaling(1./255))
#flattening the images
model_test.add(Flatten(input_shape=(256,256,3)))
#adding first hidden layer
model_test.add(Dense(64,activation='relu'))
#adding second hidden layer
model_test.add(Dense(32,activation='relu'))
#adding output layer
model_test.add(Dense(2,activation='softmax'))

In [62]:
model_test.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.01),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

In [63]:
model_test.fit(train_ds.repeat(), epochs=epochs, steps_per_epoch=50, validation_data=val_ds) 

Epoch 1/5
50/50 [==============================] - 14s 261ms/step - loss: 81.9484 - accuracy: 0.6875 - val_loss: 9.4504 - val_accuracy: 0.8169
Epoch 2/5
50/50 [==============================] - 12s 244ms/step - loss: 16.0671 - accuracy: 0.7411 - val_loss: 9.8585 - val_accuracy: 0.7606
Epoch 3/5
50/50 [==============================] - 12s 243ms/step - loss: 9.8743 - accuracy: 0.7773 - val_loss: 12.9803 - val_accuracy: 0.7042
Epoch 4/5
50/50 [==============================] - 12s 242ms/step - loss: 10.6412 - accuracy: 0.7519 - val_loss: 4.2059 - val_accuracy: 0.8099
Epoch 5/5
50/50 [==============================] - 11s 224ms/step - loss: 3.0325 - accuracy: 0.8295 - val_loss: 0.9892 - val_accuracy: 0.8521


In [64]:
model_test.evaluate(val_ds)

5/5 [==============================] - 1s 35ms/step - loss: 0.9892 - accuracy: 0.8521


[0.9891656041145325, 0.8521126508712769]

In [65]:
model_test.evaluate(test_ds)

1/1 [==============================] - 0s 58ms/step - loss: 4.6884 - accuracy: 0.6250


[4.688446044921875, 0.625]

$\underline{Remarque :}$  

On peut avoir de meilleur résultat en ajoutant plus de nombre de couches, de neurones et de learning rate dans les hyperparamètres. Plus on ajoutera et plus le temps de calcul sera long et plus le modèle aura des chances d'avoir des performances meilleurs. On obtient un résultat d'accuracy de 62.5% sur le test, ce résultat peut être amélioré.